# Applied Data Science capstone project (Week 3) - Complete
# Segmenting and Clustering Neighborhoods in Toronto

### Part 1 : loading Toronto postal codes and structuring data in dataframe

In [112]:
import numpy as np
import pandas as pd

#### loading data to dataframe

In [113]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

#### Size of the initial dataframe

In [114]:
df.shape

(287, 3)

In [115]:
sum(df['Borough']=='Not assigned')

77

#### removing the rows where the name of Borough is not assigned

In [116]:
df=df[df['Borough']!='Not assigned']

In [117]:
df

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


#### Grouping rows with the same postcode

In [118]:
df=df.groupby(['Postcode']).agg({'Borough': 'first','Neighborhood':lambda x: ", ".join(x)}).reset_index()

In [119]:
df

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


#### Assigning the borough name to the neighborhood with no name

In [120]:
df[df['Neighborhood']=='Not assigned']

,Postcode,Borough,Neighborhood
93,M9A,Queen's Park,Not assigned


In [121]:
for i in range(len(df)):
    if df['Neighborhood'].iloc[i]=='Not assigned':
        df['Neighborhood'].iloc[i]=df['Borough'].iloc[i]

In [122]:
df.loc[93]

Postcode                 M9A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 93, dtype: object

#### Size the final dataframe after processing

In [123]:
df.shape

(103, 3)

### Part 2 : Getting the latitude and the longitude coordinates of each neighborhood

#### loading the latitude and longitude of postal codes

In [124]:
dfcor=pd.read_csv('http://cocl.us/Geospatial_data')

In [125]:
dfcor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [126]:
dfcor.shape

(103, 3)

#### assigning latitude and longitude to neighborhoods by merging the initial dataframe with the new one having the coordinate coordinates

In [127]:
#dfn : dataframe of neighborhoods with their coordinates
dfn=df.merge(dfcor,left_on='Postcode',right_on='Postal Code')

In [128]:
dfn.drop('Postal Code',axis=1, inplace=True)

In [129]:
dfn

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


### Part 3 : Exploring and clustering the neighborhoods in Toronto

In [130]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfn['Borough'].unique()),
        dfn.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [131]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# map rendering library
import folium 

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# library to handle requests
import requests 

# import k-means from clustering
from sklearn.cluster import KMeans

#Toronto coordinates 43.6532° N, 79.3832° W

#### Use geopy library to get the latitude and longitude values of Toronto City.

In [132]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address, timeout=3)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top

In [133]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, long, borough, neighborhood in zip(dfn['Latitude'], dfn['Longitude'], dfn['Borough'], dfn['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Using Foursquare to explore data

In [134]:
CLIENT_ID = 'VIH325HMPYVXQRBJQGF2APPQIXC5GROL4IOYIPOK11EAMZUR'
CLIENT_SECRET = 'TOEZWP5XLZJ4XUYI5OFUHPEG0TWM5SBU2IB5G5DXEDQVZPRQ'
VERSION = '20200101'


####  Explore the 'North Toronto West' Neighborhood

In [135]:
dfn[dfn['Neighborhood']=='North Toronto West']

,Postcode,Borough,Neighborhood,Latitude,Longitude
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [136]:
ntw=dfn.loc[46, 'Neighborhood']
ntw_lat, ntw_long = dfn.loc[46, 'Latitude'], dfn.loc[46, 'Longitude']

print('Latitude and longitude values of {} are {}, {}.'.format(ntw,ntw_lat,ntw_long))

Latitude and longitude values of North Toronto West are 43.7153834, -79.40567840000001.


#### Exploring the top 100 venues in North Toronto West within a radius of 500 meters.

In [137]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    ntw_lat, 
    ntw_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VIH325HMPYVXQRBJQGF2APPQIXC5GROL4IOYIPOK11EAMZUR&client_secret=TOEZWP5XLZJ4XUYI5OFUHPEG0TWM5SBU2IB5G5DXEDQVZPRQ&v=20200101&ll=43.7153834,-79.40567840000001&radius=500&limit=100'

using foursquare to get venues

In [138]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e14ffafdff8e6001b764004'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lawrence Park South',
  'headerFullLocation': 'Lawrence Park South, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 43.7198834045, 'lng': -79.39946407145794},
   'sw': {'lat': 43.7108833955, 'lng': -79.41189272854209}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51606062e4b0878cf540f4a2',
       'name': 'Barreworks',
       'location': {'address': '2576 Yonge St',
        'lat': 43.71407030751952,
        'lng': -79.40010911522093,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.7

In [139]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Convert json and structure it into a pandas dataframe.

In [140]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Barreworks,Yoga Studio,43.714070,-79.400109
1,C'est Bon,Chinese Restaurant,43.716785,-79.400406
2,Uncle Betty's Diner,Diner,43.714452,-79.400091
3,Civello Salon,Salon / Barbershop,43.715111,-79.400304
4,Nailsense,Spa,43.717467,-79.400653


In [141]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


#### Generalize the process to the others neigbhborhoos in Toronto

In [142]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

create a new dataframe for the results called *toronto_venues*.

In [143]:
toronto_venues = getNearbyVenues(names=dfn['Neighborhood'],
                                   latitudes=dfn['Latitude'],
                                   longitudes=dfn['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

check the size of the resulting dataframe

In [144]:
print(toronto_venues.shape)
toronto_venues.head()

(2238, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.594054,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.594054,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.594054,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.594054,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.594054,G & G Electronics,43.765309,-79.191537,Electronics Store


In [145]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",4,4,4,4,4,4
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",10,10,10,10,10,10
...,...,...,...,...,...,...
Willowdale West,6,6,6,6,6,6
Woburn,3,3,3,3,3,3
"Woodbine Gardens, Parkview Hill",10,10,10,10,10,10


## Analyzing Each Neighborhood

#### transforming dataframe on one hot encoding

In [146]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [147]:
toronto_onehot.shape

(2238, 271)

#### grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [148]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00


In [149]:
toronto_grouped.shape

(99, 271)

#### Printing each neighborhood along with the top 5 most common venues

In [150]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.08
1               Bar  0.04
2        Steakhouse  0.04
3              Café  0.04
4  Sushi Restaurant  0.03


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4   Mediterranean Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
              venue  freq
0              Park  0.25
1        Playground  0.25
2       Coffee Shop  0.25
3  Asian Restaurant  0.25
4       Yoga Studio  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store   0.2
1              Pharmacy   0.1
2  Fast Food Restaurant   0.1
3           Pizza Place   0.1
4   Fried Chicken Joint   0.1


----Alderwood, Long Branch----
                venue  fr

                venue  freq
0          Beer Store  0.10
1                 Gym  0.10
2         Coffee Shop  0.10
3    Asian Restaurant  0.10
4  Chinese Restaurant  0.05


----Forest Hill North, Forest Hill West----
                      venue  freq
0                      Park  0.25
1             Jewelry Store  0.25
2          Sushi Restaurant  0.25
3                     Trail  0.25
4  Mediterranean Restaurant  0.00


----Glencairn----
                 venue  freq
0          Pizza Place  0.25
1  Japanese Restaurant  0.25
2               Bakery  0.25
3                  Pub  0.25
4               Market  0.00


----Guildwood, Morningside, West Hill----
               venue  freq
0  Electronics Store  0.12
1        Pizza Place  0.12
2                Spa  0.12
3     Breakfast Spot  0.12
4       Intersection  0.12


----Harbord, University of Toronto----
                 venue  freq
0                 Café  0.17
1                  Bar  0.06
2           Restaurant  0.06
3  Japanese Restaurant  0

4  Furniture / Home Store  0.05


----The Junction North, Runnymede----
            venue  freq
0     Pizza Place   0.2
1        Bus Line   0.2
2  Breakfast Spot   0.2
3         Brewery   0.2
4   Grocery Store   0.2


----The Kingsway, Montgomery Road, Old Mill North----
                      venue  freq
0                      Park   0.5
1                     River   0.5
2               Yoga Studio   0.0
3  Mediterranean Restaurant   0.0
4         Mobile Phone Shop   0.0


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.11
1       Burger Joint  0.11
2        Yoga Studio  0.06
3               Bank  0.06
4                Gym  0.06


----Victoria Village----
                   venue  freq
0  Portuguese Restaurant  0.17
1            Pizza Place  0.17
2           Intersection  0.17
3      French Restaurant  0.17
4            Coffee Shop  0.17


----Westmount----
                venue  freq
0         Pizza Place  0.25
1         Coffee Shop  0.12
2        Intersect

#### converting results into a *pandas* dataframe

In [151]:
# function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [152]:
# creating a new dataframe and displaying the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Bar,Café,Burger Joint,Hotel,Sushi Restaurant,Thai Restaurant,Restaurant,Asian Restaurant
1,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Coffee Shop,Asian Restaurant,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Fast Food Restaurant,Beer Store,Discount Store,Japanese Restaurant,Sandwich Place,Dog Run
4,"Alderwood, Long Branch",Pizza Place,Pub,Pharmacy,Sandwich Place,Athletics & Sports,Pool,Skating Rink,Coffee Shop,Gym,Comfort Food Restaurant


## Cluster Neighborhoods

Running *k*-means to cluster the neighborhood into 5 clusters.

In [192]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 0, 0, 0, 0, 0])

creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [199]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = dfn

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged=toronto_merged.dropna(subset=['Cluster Labels'])

visualizing the resulting clusters

In [200]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyzing Clusters

#### Cluster 1

In [202]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,Bar,History Museum,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
2,Scarborough,0.0,Rental Car Location,Mexican Restaurant,Spa,Breakfast Spot,Intersection,Medical Center,Electronics Store,Pizza Place,Concert Hall,Dim Sum Restaurant
3,Scarborough,0.0,Coffee Shop,Korean Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
4,Scarborough,0.0,Fried Chicken Joint,Bank,Hakka Restaurant,Lounge,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Gas Station,Discount Store
5,Scarborough,0.0,Playground,Business Service,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore
...,...,...,...,...,...,...,...,...,...,...,...,...
97,North York,0.0,Baseball Field,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
99,Etobicoke,0.0,Pizza Place,Middle Eastern Restaurant,Chinese Restaurant,Coffee Shop,Sandwich Place,Discount Store,Intersection,Dessert Shop,Dim Sum Restaurant,Diner
100,Etobicoke,0.0,Mobile Phone Shop,Bus Line,Sandwich Place,Park,Colombian Restaurant,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
101,Etobicoke,0.0,Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Fast Food Restaurant,Beer Store,Discount Store,Japanese Restaurant,Sandwich Place,Dog Run


#### Cluster 2

In [203]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


#### Cluster 3

In [204]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,2.0,Golf Course,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


#### Cluster 4

In [205]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,3.0,Park,Coffee Shop,Asian Restaurant,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
23,North York,3.0,Park,Convenience Store,Bank,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Department Store
25,North York,3.0,Park,Food & Drink Shop,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Falafel Restaurant
30,North York,3.0,Park,Airport,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
40,East York,3.0,Park,Convenience Store,Coffee Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
48,Central Toronto,3.0,Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
50,Downtown Toronto,3.0,Park,Trail,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
64,Central Toronto,3.0,Park,Trail,Jewelry Store,Sushi Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
74,York,3.0,Park,Fast Food Restaurant,Market,Women's Store,Golf Course,Gluten-free Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Greek Restaurant
90,Etobicoke,3.0,River,Park,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop


#### Cluster 5

In [206]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
79,North York,4.0,Park,Basketball Court,Bakery,Construction & Landscaping,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


The End!